Primeira entrega
Para essa entrega focaremos somente nas etapas de: 
- Escolher uma API da lista disponível; 
- Fazer a extração de pelo menos 3 tabelas;
- riar um alerta de erro usando a função alerta() do desafio da aula Fundamentos de programação III.


In [1]:
import requests
import pandas as pd
import time
from plyer import notification

# Função para exibir alertas
def alerta(titulo, mensagem):
    notification.notify(
        title=titulo,
        message=mensagem,
        app_name="Sistema de Alertas para ETL",
        timeout=10
    )

# URL da API
url = "https://restcountries.com/v3.1/all"

try:
    # Marca o início do processo
    start_time = time.time()

    # ETAPA 1: EXTRAÇÃO
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
    else:
        alerta("Falha na Extração", f"Código de status {response.status_code}")
        exit()

    # Verificação de dados ausentes
    if not data:
        alerta("Dados Ausentes", "Nenhum dado retornado pela API.")
        exit()

    # ETAPA 2: TRANSFORMAÇÃO
    try:
        # Tabela 1: Países e Continentes
        continentes = [{"País": country.get("name", {}).get("common", "N/A"),
                        "Continente": country.get("region", "N/A")}
                       for country in data]
        df_continentes = pd.DataFrame(continentes)

        # Tabela 2: Países e Línguas
        linguas = []
        for country in data:
            nome_pais = country.get("name", {}).get("common", "N/A")
            if "languages" in country:
                for lingua in country["languages"].values():
                    linguas.append({"País": nome_pais, "Língua": lingua})
        df_linguas = pd.DataFrame(linguas)

        # Tabela 3: Países e Moedas
        moedas = []
        for country in data:
            nome_pais = country.get("name", {}).get("common", "N/A")
            if "currencies" in country:
                for moeda, detalhes in country["currencies"].items():
                    moedas.append({
                        "País": nome_pais,
                        "Moeda": moeda,
                        "Nome da Moeda": detalhes.get("name", "N/A")
                    })
        df_moedas = pd.DataFrame(moedas)

    except Exception as e:
        alerta("Erro de Transformação", f"Erro ao processar dados: {str(e)}")
        exit()

    # ETAPA 3: CARGA (Simulada com print)
    try:
        print("Tabela 1: Países e Continentes")
        print(df_continentes.head())

        print("\nTabela 2: Países e Línguas")
        print(df_linguas.head())

        print("\nTabela 3: Países e Moedas")
        print(df_moedas.head())
    except Exception as e:
        alerta("Erro de Carga", f"Erro ao carregar os dados: {str(e)}")
        exit()

    # Verificação de atraso na carga
    elapsed_time = time.time() - start_time
    if elapsed_time > 5:  # Ajuste o limite conforme necessário
        alerta("Atraso na Carga", f"O processo levou {elapsed_time:.2f} segundos.")

except Exception as e:
    alerta("Erro Geral", f"Erro inesperado: {str(e)}")


Tabela 1: Países e Continentes
            País Continente
0  South Georgia  Antarctic
1        Grenada   Americas
2    Switzerland     Europe
3   Sierra Leone     Africa
4        Hungary     Europe

Tabela 2: Países e Línguas
            País        Língua
0  South Georgia       English
1        Grenada       English
2    Switzerland        French
3    Switzerland  Swiss German
4    Switzerland       Italian

Tabela 3: Países e Moedas
            País Moeda             Nome da Moeda
0  South Georgia   SHP        Saint Helena pound
1        Grenada   XCD  Eastern Caribbean dollar
2    Switzerland   CHF               Swiss franc
3   Sierra Leone   SLL      Sierra Leonean leone
4        Hungary   HUF          Hungarian forint
